# Notebook #1: Captura de datos y limpieza

In [1]:
# Importamos liberías

from tqdm import tqdm  # Usar barras de progreso
import requests  # Trabajar con APIs
from time import sleep  # Funciones con time (pausas, espera)
import pandas as pd  # Trabajar con DataFrames para análisis de datos
import geopandas as gpd
from shapely.geometry import Point
import numpy as np # Trabajar con arrays y operaciones matemáticas avanzadas
from IPython.display import display  # Mostrar salidas de manera más clara en entornos interactivos (como Jupyter)
import time  # Funciones relacionadas con el tiempo
import datetime  # Obtener la fecha y hora actuales
import os  # Interactuar con el sistema operativo, como rutas y variables de entorno
import dotenv  # Manejo de archivos .env para cargar tokens y claves
dotenv.load_dotenv()  # Cargar variables de entorno desde un archivo .env
import json

from googletrans import Translator

from geopy.geocoders import Nominatim

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.by import By # By permite localizar elementos web usando diferentes estrategias de búsqueda (ID, CSS_SELECTOR, XPATH, etc.)
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait  # Esperas explícitas para que ciertos elementos sean visibles o interactuables.
from selenium.webdriver.support import expected_conditions as EC  # Condiciones esperadas que ayudan a realizar esperas explícitas en Selenium.
from selenium.common.exceptions import NoSuchElementException  # Excepciones comunes de selenium, como cuando no se encuentra un elemento.
from bs4 import BeautifulSoup  # Herramienta para extraer y analizar datos de páginas HTML.
import re



/Users/davidfranco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Librerías para tratamiento de datos

import pandas as pd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los DFs
import numpy as np

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_funciones as sf #Archivo .py donde encontraremos todas nuestras funciones.
#from src import soporte_variables as sv

# Librería para trabajar con archivos pickle
import pickle

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

## API AirBnB

In [3]:
#resultados_airbnb = sf.consulta_airbnbs("Madrid", "2025-01-31", "2025-02-02", 9)

In [4]:
#with open("../datos/origen/airbnb.json", "w") as json_file:
#    json.dump(resultados_airbnb, json_file, indent=4)

In [5]:
with open("../datos/origen/airbnb.json", 'r') as file:
    resultados_airbnb = json.load(file)

In [6]:
df_airbnb = sf.dataframe_airbnb(resultados_airbnb)

In [7]:
df_airbnb.shape

(320, 4)

- Usaremos la función `traducir_es`, que recibe un texto y lo traduce a español haciendo uso de Google Translate, y la aplicaremos a la columna "Descripción", para traducir su contenido a español.

In [8]:
#df_airbnb["Descripción"] = df_airbnb["Descripción"].apply(sf.traducir_es)

In [9]:
df_airbnb.sample(3)

,Latitud,Longitud,Descripción,Precio Total
120,40.46984,-3.70078,Room in madrid with good family atmosphere!,78
115,40.41766,-3.70692,*** Study Center Plaza Mayor-Sol Wifi A/C***,284
16,40.41298,-3.70963,Piso Plaza Mayor-La Latina-A/C-Balcón-Ascensor,273


- Convertimos la latitud y la longitud a geopuntos, cambiamos el formato de CRS (Coordinate Reference System) para que estandarizar el formato con los datos obtenidos de los municipios de Madrid, y guardamos en un archivo de tipo `geojason`.

In [10]:
gdf_airbnb = gpd.GeoDataFrame(df_airbnb, geometry=gpd.points_from_xy(df_airbnb.Longitud, df_airbnb.Latitud))
gdf_airbnb.crs = "EPSG:4326"
#gdf_airbnb.to_file('../datos/origen/airbnb.geojson', driver='GeoJSON')

In [11]:
gdf_distritos = gpd.read_file("../datos/origen/madrid-districts.geojson")

In [12]:
gdf_distritos = gdf_distritos.rename(columns={"name":"Distrito","cartodb_id":"ID_Distrito"})
gdf_distritos.drop(columns= ["created_at", "updated_at"], inplace=True)

In [13]:
gdf_sjoin = gpd.sjoin(gdf_airbnb, gdf_distritos, how="inner", predicate="within")
gdf_sjoin = gdf_sjoin.drop(columns="index_right")

In [14]:
gdf_sjoin.head(1)

,Latitud,Longitud,Descripción,Precio Total,geometry,Distrito,ID_Distrito
0,40.411549,-3.697992,Cozy ensuite double room in the heart of Madrid,186,POINT (-3.69799 40.41155),Centro,1


In [15]:
gdf_sjoin_final = gdf_sjoin[["ID_Distrito", "Precio Total", "Descripción", "Latitud", "Longitud", "geometry"]]

In [16]:
gdf_sjoin_final.sample(2)

,ID_Distrito,Precio Total,Descripción,Latitud,Longitud,geometry
110,2,361,**Center Luxurious Apartment. Museum Mile Wifi...,40.408260,-3.692910,POINT (-3.69291 40.40826)
160,1,292,"Apartment Centro, Madrid. WIFI. A/C.",40.419794,-3.706152,POINT (-3.70615 40.41979)


In [17]:
#gdf_sjoin_final.to_file('../datos/finales/airbnb.geojson', driver='GeoJSON')
#gdf_sjoin.to_file('../datos/finales/airbnb.shp')

## Creación del DF Distritos
- En él se listan los distritos de Madrid junto con los códigos asignados por el ayuntamiento.
- Guardamos en .CSV.

In [18]:
df_distritos = gdf_distritos
#df_distritos.to_file('../datos/finales/distritos.geojson', driver='GeoJSON')
df_distritos

,Distrito,ID_Distrito,geometry
0,Centro,1,"MULTIPOLYGON (((-3.69185 40.40853, -3.69189 40..."
1,Arganzuela,2,"MULTIPOLYGON (((-3.70258 40.40638, -3.70166 40..."
2,Retiro,3,"MULTIPOLYGON (((-3.66279 40.4097, -3.66384 40...."
3,Salamanca,4,"MULTIPOLYGON (((-3.65809 40.43945, -3.65828 40..."
4,Chamartin,5,"MULTIPOLYGON (((-3.67231 40.48388, -3.67237 40..."
5,Tetuan,6,"MULTIPOLYGON (((-3.69633 40.47572, -3.69619 40..."
6,Chamberi,7,"MULTIPOLYGON (((-3.68991 40.44737, -3.69048 40..."
7,Fuencarral-El Pardo,8,"MULTIPOLYGON (((-3.64131 40.63922, -3.64118 40..."
8,Moncloa-Aravaca,9,"MULTIPOLYGON (((-3.79973 40.47063, -3.79887 40..."
9,Latina,10,"MULTIPOLYGON (((-3.7213 40.41256, -3.72051 40...."


## WebScraping de Redpiso

In [19]:
#sopas_redpiso = sf.scraping_alquileres_redpiso(50)

In [20]:
#with open('../datos/origen/sopas_redpiso.pkl', 'wb') as file:
#    pickle.dump(sopas_redpiso, file)

In [21]:
with open('../datos/origen/sopas_redpiso.pkl', 'rb') as file:
    sopas_redpiso = pickle.load(file)

In [22]:
df_redpiso = sf.dataframe_redpiso(sopas_redpiso)

- Vemos que, hemos obtenido un DF con un total de 599 viviendas en alquiler.

In [23]:
df_redpiso.shape

(600, 2)

- Procederemos ahora a limpiar los datos:
1. Aplicaremos la función `extraer_distrito` a la columna descripción. Esta función aplica un patrón de Regex al string para obtener únicamente el distrito. Si no lo encuentra, devuelve "Distrito no identificado". Limpiaremos esos registros posteriormente.
2. En la columna precio sustituimos los puntos, los signos de euro y los strings "a consultar".
3. Homogenizamos los nombres de los distritos eliminando las tildes y los nombres generales. 

In [24]:
df_redpiso["Distrito"] = df_redpiso['Descripción'].apply(sf.extraer_distrito)
df_redpiso["Descripción"] = df_redpiso["Descripción"].str.title()
df_redpiso["Precio"] = df_redpiso["Precio"].str.replace("A consultar","0")
df_redpiso["Precio"] = df_redpiso["Precio"].str.replace(".","")
df_redpiso["Precio"] = df_redpiso["Precio"].str.replace(" €","").astype(int)
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Villa de Vallecas-Ensanche y Santa Eugenia","Puente de Vallecas")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("San Blas-Canillejas","San Blas")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Vicálvaro-Ambroz-Centro-Valdebernardo-Valderribas","Vicálvaro")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Chamberí","Chamberi")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Chamartín","Chamartin")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Vicálvaro","Vicalvaro")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Tetuán","Tetuan")

- Tras aplicar el patrón de Regex, nos hemos quedado con 20 entradas sin identificar. Asignaremos esos distritos manualmente y comprobaremos que se hayan eliminado.

In [25]:
df_redpiso["Distrito"][0] = "Villaverde"
df_redpiso["Distrito"][13] = "Moncloa-Aravaca"
df_redpiso["Distrito"][39] = "Moncloa-Aravaca"
df_redpiso["Distrito"][112] = "Moncloa-Aravaca"
df_redpiso["Distrito"][187] = "Villaverde"
df_redpiso["Distrito"][235] = "Centro"
df_redpiso["Distrito"][257] = "Puente de Vallecas"
df_redpiso["Distrito"][273] = "Centro"
df_redpiso["Distrito"][288] = "Moncloa-Aravaca"
df_redpiso["Distrito"][380] = "Usera"
df_redpiso["Distrito"][387] = "Moncloa-Aravaca"
df_redpiso["Distrito"][446] = "Centro"
df_redpiso["Distrito"][464] = "Moncloa-Aravaca"
df_redpiso["Distrito"][493] = "Centro"
df_redpiso["Distrito"][501] = "Centro"
df_redpiso["Distrito"][527] = "Chamberi"
df_redpiso["Distrito"][551] = "Chamartin"
df_redpiso["Distrito"][568] = "Moncloa-Aravaca"
df_redpiso["Distrito"][591] = "Moncloa-Aravaca"
df_redpiso["Distrito"][592] = "Moncloa-Aravaca"

In [26]:
df_redpiso[df_redpiso["Distrito"].str.contains("Distrito no identificado", case=False, na=False)]

,Descripción,Precio,Distrito


- Comprobamos nuevamente que los nombres de los distritos son consistentes y no tenemos registros inesperados.

In [27]:
df_redpiso["Distrito"].unique()

array(['Villaverde', 'Chamberi', 'Salamanca', 'Ciudad Lineal',
       'Puente de Vallecas', 'Barajas', 'San Blas', 'Arganzuela',
       'Tetuan', 'Moncloa-Aravaca', 'Latina', 'Hortaleza',
       'Fuencarral-El Pardo', 'Retiro', 'Centro', 'Carabanchel',
       'Chamartin', 'Moratalaz', 'Usera', 'Vicalvaro'], dtype=object)

- Eliminamos los registros donde el precio del alquiler sea cero, debido a que distorsionarían el análisis.

In [28]:
df_redpiso["Precio"].value_counts()

Precio
1100    53
1200    51
0       43
850     42
1000    40
        ..
2350     1
550      1
690      1
720      1
1480     1
Name: count, Length: 83, dtype: int64

In [29]:
df_redpiso = df_redpiso[
    (df_redpiso["Precio"] != 0)
]

- Tras eliminar los registros, nos quedamos con 556 pisos repartidos en 20 distritos, de un total de 21 que tenemos en Madrid.

In [30]:
df_redpiso["Distrito"].value_counts()

Distrito
Chamberi               74
Chamartin              69
Centro                 62
Fuencarral-El Pardo    47
Hortaleza              44
Puente de Vallecas     43
Arganzuela             35
Salamanca              33
Latina                 31
Moncloa-Aravaca        26
Tetuan                 25
Retiro                 17
Carabanchel            15
San Blas               11
Villaverde              8
Ciudad Lineal           6
Vicalvaro               6
Barajas                 2
Moratalaz               2
Usera                   1
Name: count, dtype: int64

- Para asignar el ID a cada municipio, realiaremos un merge con el dataframe de distritos. Reordenaremos las columnas y lo guardaremos en un archivo CSV.

In [31]:
df_redpiso_merge = df_redpiso.merge(df_distritos, how="inner", left_on="Distrito", right_on="Distrito")

In [32]:
df_redpiso_merge.drop(columns = ["Distrito", "geometry"], inplace=True)
df_redpiso_merge = df_redpiso_merge[["ID_Distrito", "Descripción", "Precio"]]
df_redpiso_merge.head(5)

,ID_Distrito,Descripción,Precio
0,17,"Piso En Alquiler En Villaverde, Madrid, Madrid",850
1,7,"Piso En Alquiler En Calle Cristobal Bordiu, Rí...",1100
2,4,"Apartamento En Alquiler En Calle Fundadores, F...",2000
3,15,"Piso En Alquiler En Calle Pepe Isbert, Pueblo ...",744
4,13,Estudio En Alquiler En Calle Embalse De Navace...,750


In [ ]:
#df_redpiso_merge.to_csv("../datos/finales/redpiso.csv")

## API Idealista

In [34]:
#resultados_idealista = sf.consulta_idealista("0-EU-ES-28-07-001-079", "Madrid", 10)

In [35]:
#with open("../datos/origen/idealista.json", "w") as json_file:
#    json.dump(resultados_idealista, json_file, indent=4)

In [36]:
with open("../datos/origen/idealista.json", 'r') as file:
    resultados_idealista = json.load(file)

In [37]:
df_idealista = sf.dataframe_idealista(resultados_idealista)

In [ ]:
df_idealista["Tipo"] = df_idealista["Tipo"].apply(sf.traducir_es)
df_idealista["Dirección"] = df_idealista["Dirección"].str.title()
df_idealista["Descripción"] = df_idealista["Descripción"].str.title()

In [51]:
df_idealista.head(2)

,Latitud,Longitud,Precio,Tipo,Planta,Tamaño,Habitaciones,Baños,Dirección,Descripción
0,40.432201,-3.714278,1350.0,departamento,1,73.0,2,1,"Calle De Andres Mellado, 18",Piso Con Una Habitación Con Cama De Matrimonio...
1,40.414669,-3.705414,2300.0,departamento,5,95.0,2,2,Calle Del Marqués Viudo De Pontejos,S C A L A | Properties Presenta Ático Amuebl...


In [ ]:
gdf_idealista = gpd.GeoDataFrame(df_idealista, geometry=gpd.points_from_xy(df_idealista.Longitud, df_idealista.Latitud))
gdf_idealista.crs = "EPSG:4326"
#gdf_idealista.to_file('../datos/origen/idealista.geojson', driver='GeoJSON')

In [53]:
gdf_sjoin2 = gpd.sjoin(gdf_idealista, gdf_distritos, how="inner", predicate="within")
gdf_sjoin2 = gdf_sjoin2.drop(columns="index_right")


In [54]:
gdf_sjoin2_final = gdf_sjoin2[['ID_Distrito', 'Precio', 'Tipo', 'Planta', 'Tamaño',
       'Habitaciones', 'Baños', 'Dirección', 'Descripción', 'Latitud', 'Longitud', 'geometry']]

In [56]:
gdf_sjoin2_final.sample(2)

,ID_Distrito,Precio,Tipo,Planta,Tamaño,Habitaciones,Baños,Dirección,Descripción,Latitud,Longitud,geometry
335,6,1750.0,departamento,2,57.0,2,1,"Calle García Cea, 4",Apartamento De Alquiler Temporal (Desde 32 Día...,40.461156,-3.698587,POINT (-3.69859 40.46116)
194,16,1600.0,departamento,3,112.0,3,2,"Calle Dionisio Inca Yupanqui, 45",Piso Reformado A Estrenar Con Vistas Despejada...,40.461709,-3.629420,POINT (-3.62942 40.46171)


In [ ]:
#gdf_sjoin2_final.to_file('../datos/finales/idealista.geojson', driver='GeoJSON')